# Using the ``fancytypes.Checkable`` class #

## A NOTE BEFORE STARTING ##

Since the ``fancytypes`` git repository tracks this notebook under its original
basename ``using_Checkable_class.ipynb``, we recommend that you copy the
original notebook and rename it to any other basename that is not one of the
original basenames that appear in the ``<root>/examples`` directory before
executing any of the notebook cells below, where ``<root>`` is the root of the
``fancytypes`` repository. This way you can explore the notebook by executing
and modifying cells without changing the original notebook, which is being
tracked by git.

## Import necessary modules ##

In [ ]:
# For general array handling and constructing random number generators.
import numpy as np



# The library that is the subject of this demonstration.
import fancytypes

## Introduction ##

In this notebook, we use the ``fancytypes.Checkable`` class to define a class of
"slice shufflers", which we define as objects that can shuffle/re-order the
elements in a slice of a given array. This is a somewhat contrived example use
of the ``fancytypes.Checkable`` class, however it is simple and complete. We also
test the exception-raising features of the ``fancytypes.Checkable`` class.

You can find the documentation for the ``fancytypes.Checkable`` class
[here](https://mrfitzpa.github.io/fancytypes/_build/html/_autosummary/fancytypes.Checkable.html). It
is recommended that you consult the documentation of this class as you explore
the notebook.


## Defining the ``SliceShuffler`` class ##

We define the ``SliceShuffler`` class as a subclass of the
``fancytypes.Checkable`` class.

In [ ]:
# Define the validation and conversion functions.
def _check_and_convert_slice_obj(ctor_params):
    slice_obj = ctor_params["slice_obj"]
    if not isinstance(slice_obj, slice):
        err_msg = ("The object ``slice_obj`` must be of type `slice`.")
        raise TypeError(err_msg)

    return slice_obj  # No conversion for this parameter/attribute.

def _check_and_convert_seed(ctor_params):
    seed = ctor_params["seed"]
    
    if seed is not None:
        try:
            seed = float(seed)
            if seed.is_integer():
                seed = int(seed)
            else:
                raise TypeError
        except:
            err_msg = ("The object ``seed`` must be an integer or `NoneType`.")
            raise TypeError(err_msg)
            
    return seed  # Conversion is implied under some circumstances.



# Define the ``SliceShuffler`` class.
class SliceShuffler(fancytypes.Checkable):
    def __init__(self, slice_obj, seed):
        ctor_params = {"slice_obj": slice_obj, "seed": seed}
        validation_and_conversion_funcs = \
            {"slice_obj": _check_and_convert_slice_obj,
             "seed": _check_and_convert_seed}
        fancytypes.Checkable.__init__(self,
                                      ctor_params,
                                      validation_and_conversion_funcs)

        # ``core_attrs`` are the "core attributes" of the instance.
        seed = self.core_attrs["seed"]
        self._random_generator = np.random.default_rng(seed)

        return None



    def shuffle(self, array):
        try:
            array = np.array(array)
        except:
            err_msg = ("The object ``array`` must be array-like.")
            raise TypeError(err_msg)
            
        slice_obj = self.core_attrs["slice_obj"]
        array_slice = array[slice_obj]
        self._random_generator.shuffle(array_slice)
        array[slice_obj] = array_slice

        return array

## Using the ``SliceShuffler`` class ##

Construct a valid instance of the ``SliceShuffler`` class and print the core attributes.

In [ ]:
slice_shuffler = SliceShuffler(slice_obj=slice(None, 6, 1), seed=5.0)

print(slice_shuffler.core_attrs)

Shuffle an array using the class.

In [ ]:
array = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
shuffled_array = slice_shuffler.shuffle(array)
shuffled_array

Try constructing instances of the ``SliceShuffler`` class with invalid
construction parameters.

In [ ]:
slice_shuffler = SliceShuffler(slice_obj=3, seed=5.0)

In [ ]:
slice_shuffler = SliceShuffler(slice_obj=slice(None, 6, 1), seed="foo")

Note that being a direct subclass of the ``fancytypes.Checkable`` class, the
``SliceShuffler`` class enforces validation upon construction, but it does not
support updatable core attributes, pre-serialization, de-pre-serialization.

The ``fancytypes.Updatable`` class supports updatable core attributes, and
validation upon constructing or updating instances.

The ``fancytypes.PreSerializable`` class supports pre-serialization,
de-serialization, and enforces validation upon constructing instances.

The ``fancytypes.PreSerializableAndUpdatable`` class supports pre-serialization,
de-serialization, updatable core attributes, and enforces validation upon
constructing or updating instances.

## Testing the exception-raising features of the ``fancytypes.Checkable`` class ##

Each cell below tests a different exception raising feature.

In [ ]:
ctor_params = {"param_1": 1, "param_2": None}

def _check_and_convert_param_1(ctor_params):
    return ctor_params["param_1"]

def _check_and_convert_param_3(ctor_params):
    return ctor_params["param_3"]

validation_and_conversion_funcs = \
    {"param_1": _check_and_convert_param_1,
     "param_3": _check_and_convert_param_3}

fancytypes.Checkable(ctor_params, validation_and_conversion_funcs)

In [ ]:
ctor_params = {"param_1": 1, "param_2": None}

def _check_and_convert_param_1(ctor_params):
    return ctor_params["param_1"]

def _check_and_convert_param_3(ctor_params):
    return ctor_params["param_3"]

validation_and_conversion_funcs = \
    {"param_1": _check_and_convert_param_1, "param_2": 5}

fancytypes.Checkable(ctor_params, validation_and_conversion_funcs)